In [1]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import *

In [2]:
def load_images_and_label(image_paths, target_size=(64,64)):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path,
                         target_size = target_size)
        image = img_to_array(image)

        label = image_path.split(os.path.sep)[-2]

        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

In [3]:
def build_network(width, height, depth, classes):
    input_layer = Input(shape=(width, height, depth))

    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Flatten()(x)
    x = Dense(units=512)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.25)(x)

    x = Dense(units=classes)(x)
    output = Softmax()(x)

    return Model(input_layer, output)

In [4]:
def plot_model_history(model_history, metric, plot_name):
    plt.style.use('seaborn-darkgrid')
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({'Model': model_history}, metric=metric)
    plt.title(f'{metric.upper()}')
    plt.ylim([0, 1])
    plt.savefig(f'{plot_name}.png')
    plt.close()

In [5]:
SEED = 999
np.random.seed(SEED)

In [6]:
base_path = ('C:/Users/hp/Documents/DATA/caltech-101/101_ObjectCategories/*/*.jpg')
image_pattern = str(base_path)
image_paths = [*glob.glob(image_pattern)]
image_paths = [p for p in image_paths if
               p.split(os.path.sep)[-2] !=
               'BACKGROUND_Google']

In [7]:
classes = {p.split(os.path.sep)[-2] for p in image_paths}

In [8]:
X, y = load_images_and_label(image_paths)
x = X.astype('float') / 255.0
y = LabelBinarizer().fit_transform(y)

In [9]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [10]:
EPOCHS = 1
BATCH_SIZE = 64
model = build_network(64, 64, 3, len(classes))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)
result = model.evaluate(X_test, y_test)
print(f'Test accuracy: {result[1]}')
plot_model_history(history, 'accuracy', 'normal')

55/55 [==============================] - 4s 76ms/step - loss: 2.6981 - accuracy: 0.4257
Test accuracy: 0.42569124698638916


C:\Users\hp\AppData\Local\Temp\ipykernel_11400\3768594648.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')
c:\Users\hp\anaconda3\lib\site-packages\tensorflow_docs\plots\__init__.py:111: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  plt.xlim(


In [11]:
model = build_network(64, 64, 3, len(classes))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
augmentor = ImageDataGenerator(horizontal_flip=True,
                               rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               shear_range=0.2,
                               zoom_range=0.2,
                               fill_mode='nearest')
train_generator = augmentor.flow(X_train, y_train, BATCH_SIZE)
hist = model.fit(train_generator, 
                 steps_per_epoch=len(X_train) // BATCH_SIZE,
                 validation_data=(X_test, y_test),
                 epochs=EPOCHS)

result = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {result[1]}')
plot_model_history(hist, 'accuracy', 'augmented')

55/55 [==============================] - 4s 73ms/step - loss: 5.1891 - accuracy: 0.1843
Test Accuracy: 0.18433180451393127


C:\Users\hp\AppData\Local\Temp\ipykernel_11400\3768594648.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')
c:\Users\hp\anaconda3\lib\site-packages\tensorflow_docs\plots\__init__.py:111: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  plt.xlim(
